In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import janitor

In [50]:
ingredients = pd.read_csv("cake_ingredient_code.csv").clean_names()

In [51]:
cakes = pd.read_csv("cake-ingredients-1961.csv").clean_names()

In [52]:
cakes = cakes.melt(id_vars="cake", var_name="ingredients", value_name="amount") # get data into tidy format

In [34]:
cakes

,cake,ingredients,amount
0,Angel,ae,0.25
1,Babas au Rhum,ae,NaN
2,Sweet Chocolate,ae,NaN
3,Buche de Noel,ae,NaN
4,Cheesecake,ae,NaN
...,...,...,...
607,Strawberry Roll,zh,NaN
608,Savarin,zh,NaN
609,Banana Shortcake,zh,NaN
610,Strawberry Shortcake,zh,NaN


In [53]:
ingredients = ingredients.rename(columns={"column1":"code", "column2":"ingredient", "column3":"measure"})
# columns named incorrectly

In [54]:
ingredients = ingredients.drop(0) # remove first row which duplicates column names

In [55]:
ingredients["code"] = ingredients["code"].str.lower() # make the ingredients codes lowercase

In [56]:
cakes_ingredients = cakes.merge(ingredients, left_on="ingredients", right_on="code") 
#combine the tables on ingredient codes

In [57]:
cakes_ingredients= cakes_ingredients.drop(columns=["code", "ingredients"]) #remove unnecessary columns

In [58]:
cakes_ingredients

,cake,amount,ingredient,measure
0,Angel,0.25,Almond essence,teaspoon
1,Babas au Rhum,NaN,Almond essence,teaspoon
2,Sweet Chocolate,NaN,Almond essence,teaspoon
3,Buche de Noel,NaN,Almond essence,teaspoon
4,Cheesecake,NaN,Almond essence,teaspoon
...,...,...,...,...
607,Strawberry Roll,NaN,Zwiebach,ounce
608,Savarin,NaN,Zwiebach,ounce
609,Banana Shortcake,NaN,Zwiebach,ounce
610,Strawberry Shortcake,NaN,Zwiebach,ounce


In [91]:
cakes_ingredients.isna().sum()  #lots of NAs

cake            0
amount        456
ingredient      0
measure        18
dtype: int64

In [92]:
cakes_ingredients["amount"] = cakes_ingredients["amount"].fillna(0)

In [93]:
cakes_ingredients["measure"] = cakes_ingredients["measure"].fillna("unknown")

In [94]:
cakes_ingredients.isna().sum() # no NAs remain

cake          0
amount        0
ingredient    0
measure       0
dtype: int64

1) Which cake has the most cocoa in it?

In [64]:
(cakes_ingredients[cakes_ingredients["ingredient"].str.contains(pat="cocoa", case=False)]
.sort_values(by="amount", ascending=False).head(1))

,cake,amount,ingredient,measure
97,One Bowl Chocolate,10.0,Cocoa,tablespoon


2) For sponge cake, how many cups of ingredients are used in total?

In [95]:
sponge_cake = cakes_ingredients[cakes_ingredients["cake"].str.contains(pat="sponge", case=False)]
#filter sponge cake

In [96]:
sponge_cake[sponge_cake["measure"].str.contains(pat="cup", case=False)]["amount"].sum()

3.5

3) How many ingredients are measured in teaspoons?

In [98]:
teaspoon_ingredients = cakes_ingredients[cakes_ingredients["measure"].str.contains(pat="teaspoon", case=False)]

In [114]:
len(teaspoon_ingredients["ingredient"].unique())

8

4) Which cake has the most ingredients?

In [142]:
cakes_with_ingredients = cakes_ingredients[cakes_ingredients["amount"] > 0] # remove the 0s

In [150]:
cakes_with_ingredients.groupby("cake").size().sort_values(ascending=False)

cake
One Bowl Chocolate       11
Babas au Rhum            11
Rum Cheesecake           10
Cheesecake               10
Strawberry Shortcake     10
Sweet Chocolate          10
Sour Cream Fudge         10
Savarin                   9
Red Devil's Food          9
Blender Cheesecake        9
Hungarian Cream           8
Crumb and Nut             8
Banana Shortcake          8
Strawberry Roll           8
Sponge                    7
Angel                     7
Spiced Pound              6
Buche de Noel             5
dtype: int64

5) Which ingredients are used only once?

In [173]:
only_once = cakes_with_ingredients.groupby("ingredient").size() == 1

In [174]:
pd.Series(only_once[only_once].index)

0            Bananas
1    Cream of tartar
2        Crushed Ice
3     Dried currants
4          Egg white
5             Nutmeg
6               Nuts
7           Zwiebach
Name: ingredient, dtype: object